In [1]:
import os
import sys
home_dir = "../../"
module_path = os.path.abspath(os.path.join(home_dir))
if module_path not in sys.path:
    sys.path.append(module_path)

import numpy as np
import pandas as pd

### Loading data

In [3]:
popu_freq_result_df = pd.read_csv(home_dir+f"models/aa_common/merged_predictions/popu_freq_analysis.csv", sep="\t")
popu_freq_result_df = popu_freq_result_df[popu_freq_result_df["class"]!="Zero-population"]
print(popu_freq_result_df.shape)
print(popu_freq_result_df.columns)
popu_freq_result_df['class'].value_counts()

(823246, 35)
Index(['snp_id', 'chrom_acc_version', 'chrom_pos', 'ref_allele', 'alt_allele',
       'prot_acc_version', 'prot_pos', 'wt', 'mut', 'wt_population',
       'mut_poulation', 'wt_freq', 'mt_freq', 'class', 'sift_pred',
       'polyphen2_HVAR_pred', 'cadd_pred', 'mvp_pred', 'metarnn_pred',
       'revel_pred', 'unirep_pred', 'protbert_pred', 'sequnet_pred',
       'esm1b_t33_650M_UR50S_pred', 'esm1v_t33_650M_UR90S_pred',
       'esm2_t33_650M_UR50D_pred', 'plus_rnn_pred', 'prottrans_bert_bfd_pred',
       'prottrans_albert_bfd_pred', 'prottrans_xlnet_uniref100_pred',
       'prottrans_t5_bfd_pred', 'prottrans_t5_uniref50_pred',
       'prottrans_t5_xl_u50_pred', 'vespal_pred', 'conservation_pred'],
      dtype='object')


Singleton     462444
Ultra-rare    314013
Rare           28622
Common         18167
Name: class, dtype: int64

In [4]:
patho_and_likelypatho_result_df = pd.read_csv(home_dir+f"models/aa_common/merged_predictions/patho_and_likelypatho_analysis.csv", sep="\t")
print(patho_and_likelypatho_result_df.shape)
print(patho_and_likelypatho_result_df.columns)
patho_and_likelypatho_result_df['class'].value_counts()

(6476, 33)
Index(['clinvar_id', 'gene_symbol', 'gene_id', 'chrom_acc_version',
       'chrom_pos', 'ref_allele', 'alt_allele', 'prot_acc_version', 'prot_pos',
       'wt', 'mut', 'class', 'sift_pred', 'polyphen2_HVAR_pred', 'cadd_pred',
       'mvp_pred', 'metarnn_pred', 'revel_pred', 'unirep_pred',
       'protbert_pred', 'sequnet_pred', 'esm1b_t33_650M_UR50S_pred',
       'esm1v_t33_650M_UR90S_pred', 'esm2_t33_650M_UR50D_pred',
       'plus_rnn_pred', 'prottrans_bert_bfd_pred', 'prottrans_albert_bfd_pred',
       'prottrans_xlnet_uniref100_pred', 'prottrans_t5_bfd_pred',
       'prottrans_t5_uniref50_pred', 'prottrans_t5_xl_u50_pred', 'vespal_pred',
       'conservation_pred'],
      dtype='object')


likely_pathogenic    4168
pathogenic           2308
Name: class, dtype: int64

In [74]:
pmd_result_df = pd.read_csv(home_dir+f"models/aa_common/merged_predictions/pmd.tsv", sep="\t")
pmd_result_df.rename(columns={"functional_effect": "class"}, inplace=True)
print(pmd_result_df.shape)
print(pmd_result_df.columns)
pmd_result_df.replace("\\N", None, inplace=True)
print(pmd_result_df["class"].value_counts())
# pmd_result_df[pd.isna(pmd_result_df['protein_acc'])] # 57773 rows does not have NP-id
# pmd_result_df[~pd.isna(pmd_result_df['uniprot_id'])]["uniprot_id"].drop_duplicates(keep="first") # 1723 uniprot-ids

def clean_crossref_ids(x):
    x = x.split()[0]
    if x[0]=="(":
        x = x[1:]
    if x[-1]==")":
        x = x[:-1]
    return x

pmd_result_df['crossref_clean'] = pmd_result_df['crossref'].apply(clean_crossref_ids)
crossref_ids = pmd_result_df['crossref_clean'].drop_duplicates(keep="first").values
crossref_ids = list(crossref_ids)
print(len(crossref_ids)) # 3898 ids
crossref_ids = ", ".join(crossref_ids)
print(len(crossref_ids.split(', ')))
crossref_ids #copy these ids to uniprotid mapping tool: 2,507 IDs were mapped to 5,561 results, 1381 ID were not mapped. download them and put it into "data/uniprot_id_mapping_tools_outputs" for next

(60306, 54)
Index(['mut_id', 'pmd_id', 'nr', 'crossref', 'uniprot_id', 'ensembl_id',
       'taxid', 'protein', 'mut_PMD', 'mut_real', 'wt', 'mut', 'prot_pos',
       'function_summarized', 'class', 'function', 'seq', 'snp_id', 'mrna_acc',
       'mrna_ver', 'mrna_pos', 'allele', 'protein_acc', 'protein_ver',
       'verified', 'chrom', 'chrom_pos', 'variation', 'variant_type',
       'ref_allele', 'alt_allele', 'pmd_nr_id', 'sift_pred',
       'polyphen2_HVAR_pred', 'cadd_pred', 'mvp_pred', 'metarnn_pred',
       'revel_pred', 'unirep_pred', 'protbert_pred', 'sequnet_pred',
       'esm1b_t33_650M_UR50S_pred', 'esm1v_t33_650M_UR90S_pred',
       'esm2_t33_650M_UR50D_pred', 'plus_rnn_pred', 'prottrans_bert_bfd_pred',
       'prottrans_albert_bfd_pred', 'prottrans_xlnet_uniref100_pred',
       'prottrans_t5_bfd_pred', 'prottrans_t5_uniref50_pred',
       'prottrans_t5_xl_u50_pred', 'vespa_pred', 'conservation_pred',
       'sift_snpdbe_pred'],
      dtype='object')
Effect       30774
No-

'MC4R_HUMAN, SYSC_YEAST, A57013, FOLC_MOUSE, NRDD_BPT4, VE6_BPV1, CFAB_HUMAN, PT1_ECOLI, GBRA1_MOUSE, PSN2_HUMAN, BDH_HUMAN, MDLB_PSEPU, CNGA2_RAT, PRDX6_HUMAN, DXR_ECOLI, DHB1_HUMAN, DIRECT1, TFDA_RALEJ, AROG_ECOLI, KDSA_ECOLI, HMCS1_CHICK, LEU3_THETH, LEU3_ECOLI, SDIS_COMTE, PDPK1_HUMAN, KAPCA_HUMAN, DIDH_RAT, 3BHS2_HUMAN, 4CL2_ARATH, THIM_BACSU, 5HT3R_MOUSE, 5HT2A_RAT, CPXJ_SACER, MGMT_HUMAN, LACG_LACLA, OGG1_HUMAN, AA2AR_HUMAN, AA1R_HUMAN, AA3R_RAT, ACES_MOUSE, ACES_HUMAN, ACES_DROME, ACHG_MOUSE, ARF3_HUMAN, CHTA_VIBCH, DTX_CORBE, AIRE_HUMAN, AAPK1_RAT, AAPK2_RAT, APEX1_HUMAN, AP2B_HUMAN, AP2A_HUMAN, AMPE_MOUSE, XPP2_PIG, MIP_HUMAN, ARGJ_YEAST, RHG06_HUMAN, AGTRA_RAT, A53651, ATP6_ECOLI, ATPA_YEAST, ATPA_ECOLI, DNLI_METTH, FTSH_ECOLI, IRK11_MOUSE, ATIF_YEAST, ACKA_METTE, ILVG_ECOLI, ILVM_ECOLI, ILV1_TOBAC, MOF_DROME, ACT_YEAST, ABP1_YEAST, ACTZ_NEUCR, KASA_STRCO, PROC_HUMAN, ACTP_ACACA, ACPS_BACSU, ACADM_HUMAN, CRK_HUMAN, PTN1_HUMAN, CRK_CHICK, PAXI_CHICK, MUTY_ECOLI, AA2BR_HUMAN, 

In [63]:
crossref_to_np_df = pd.read_csv(home_dir+f"data/uniprot_id_mapping_tools_outputs/pmd_crossref_to_np_mapping.tsv", sep="\t")
crossref_to_np_df.rename(columns={"From": "crossref_clean", "To": "prot_acc_version"}, inplace=True)
crossref_to_np_df = crossref_to_np_df[crossref_to_np_df["prot_acc_version"].apply(lambda x: x.startswith("NP_"))]
crossref_to_np_df.value_counts()

crossref_clean  prot_acc_version
1433B_HUMAN     NP_003395.1         1
P73_HUMAN       NP_001191120.1      1
                NP_005418.1         1
P85A_BOVIN      NP_777000.1         1
P85A_HUMAN      NP_001229395.1      1
                                   ..
FUR4_YEAST      NP_009577.1         1
FURIN_HUMAN     NP_001276752.1      1
                NP_001276753.1      1
                NP_002560.1         1
ZYX_HUMAN       NP_003452.1         1
Length: 3917, dtype: int64

In [65]:
pmd_merged_with_nps_df = pd.merge(left=pmd_result_df, right=crossref_to_np_df, on='crossref_clean', how="inner")


,prot_acc_version,prot_pos,wt,mut
0,NP_005903.2,271,C,Y
2,NP_005903.2,62,N,S
3,NP_005903.2,170,I,V
5,NP_005903.2,301,I,T
6,NP_005903.2,250,L,Q
...,...,...,...,...
66710,NP_001189443.1,3,Q,R
66711,NP_001189442.1,3,Q,R
66712,NP_002348.1,11,Y,D
66713,NP_001189443.1,11,Y,D


### Computing overlapping variants having ['prot_acc_version', 'prot_pos', 'wt', 'mut'] columns

In [67]:
def get_overlapping_variants(df1, df2):
    df1_seq = set(map(tuple, df1.values))
    df2_seq = set(map(tuple, df2.values))

    overlapping_set = df1_seq.intersection(df2_seq)
    len(overlapping_set)
    overlapping_df = pd.DataFrame(list(overlapping_set))
    overlapping_df.rename(columns={0:'prot_acc_version', 1:'prot_pos', 2:'wt', 3:'mut'}, inplace=True)
    return overlapping_df

In [70]:
pmd_variants_df = pmd_merged_with_nps_df[['prot_acc_version', 'prot_pos', 'wt', 'mut']].drop_duplicates(keep="first")
popu_freq_variants_df = popu_freq_result_df[['prot_acc_version', 'prot_pos', 'wt', 'mut']].drop_duplicates(keep="first")
patho_variants_df = patho_and_likelypatho_result_df[['prot_acc_version', 'prot_pos', 'wt', 'mut']].drop_duplicates(keep="first")

In [71]:
get_overlapping_variants(pmd_variants_df, popu_freq_variants_df)

,prot_acc_version,prot_pos,wt,mut
0,NP_000013.2,304,L,R
1,NP_005948.3,377,R,C
2,NP_000681.2,504,E,K
3,NP_000509.1,102,E,D
4,NP_000032.1,163,R,C
...,...,...,...,...
366,NP_000017.1,114,Y,H
367,NP_000575.1,31,E,A
368,NP_004100.1,132,D,N
369,NP_000362.1,142,V,I


In [72]:
get_overlapping_variants(pmd_variants_df, patho_variants_df)

,prot_acc_version,prot_pos,wt,mut
0,NP_000397.1,217,S,R
1,NP_000124.1,394,M,I
2,NP_000425.1,328,G,S
3,NP_000124.1,262,I,M
4,NP_000013.2,74,G,D
5,NP_000373.1,423,R,H
6,NP_000445.1,150,I,T
7,NP_000039.2,398,A,D
8,NP_000454.1,276,G,R
9,NP_000093.1,496,R,C


### Popu-freq vs pathogenicity

In [73]:
get_overlapping_variants(patho_variants_df, popu_freq_variants_df)

,prot_acc_version,prot_pos,wt,mut
0,NP_001076585.1,317,G,R
1,NP_997304.3,433,P,L
2,NP_000491.4,133,R,C
3,NP_079492.2,108,S,P
4,NP_000488.3,116,W,C
...,...,...,...,...
360,NP_004444.2,505,I,T
361,NP_002171.2,264,R,C
362,NP_000469.3,374,V,M
363,NP_004259.3,253,G,R
